In [1]:
import numpy as np
from sklearn.tree import DecisionTreeClassifier
from sklearn.datasets import fetch_mldata
from sklearn.model_selection import train_test_split

# Load MNIST dataset
mnist = fetch_mldata('MNIST original')
X = mnist.data
y = mnist.target

# Normalize pixel values
X = X / 255.0

# Split dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

/opt/anaconda3/lib/python3.11/site-packages/scipy/__init__.py:132: UserWarning: A NumPy version >=1.21.6 and <1.28.0 is required for this version of SciPy (detected version 2.0.2)
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"

A module that was compiled using NumPy 1.x cannot be run in
NumPy 2.0.2 as it may crash. To support both 1.x and 2.x
versions of NumPy, modules must be compiled with NumPy 2.0.
Some module may need to rebuild instead e.g. with 'pybind11>=2.12'.

If you are a user of the module, the easiest solution will be to
downgrade to 'numpy<2' or try to upgrade the affected module.
We expect that some modules will need time to support NumPy 2.

Traceback (most recent call last):  File "<frozen runpy>", line 198, in _run_module_as_main
  File "<frozen runpy>", line 88, in _run_code
  File "/opt/anaconda3/lib/python3.11/site-packages/ipykernel_launcher.py", line 17, in <module>
    app.launch_new_instance()
  File "/opt/anaconda3/lib/python3.11/site

AttributeError: _ARRAY_API not found

ImportError: numpy.core.multiarray failed to import

In [ ]:

class AdaBoost:
    def __init__(self, n_estimators=50, learning_rate=1.0):
        """
        Initialize the AdaBoost model.

        Parameters:
        n_estimators (int): Number of estimators (Decision Trees) to use.
        learning_rate (float): Learning rate for the model.
        """
        self.n_estimators = n_estimators
        self.learning_rate = learning_rate
        self.estimators = []
        self.weights = []

    def fit(self, X, y):
        """
        Train the AdaBoost model.

        Parameters:
        X (array-like): Feature matrix.
        y (array-like): Target vector.
        """
        n_samples = X.shape[0]
        weights = np.full(n_samples, 1 / n_samples)

        for _ in range(self.n_estimators):
            estimator = DecisionTreeClassifier()
            estimator.fit(X, y, sample_weight=weights)

            predictions = estimator.predict(X)
            errors = np.where(predictions != y, 1, 0)

            error_rate = np.dot(weights, errors) / np.sum(weights)
            if error_rate > 0.5:
                continue

            alpha = self.learning_rate * np.log((1 - error_rate) / error_rate)
            weights *= np.exp(alpha * errors)

            # Normalize weights
            weights /= np.sum(weights)

            self.estimators.append(estimator)
            self.weights.append(alpha)

    def predict(self, X):
        """
        Make predictions using the AdaBoost model.

        Parameters:
        X (array-like): Feature matrix.

        Returns:
        array-like: Predicted target vector.
        """
        predictions = np.array([estimator.predict(X) for estimator in self.estimators])
        weighted_predictions = np.dot(self.weights, predictions)
        return np.argmax(weighted_predictions, axis=0)

    def score(self, X, y):
        """
        Evaluate the AdaBoost model.

        Parameters:
        X (array-like): Feature matrix.
        y (array-like): Target vector.

        Returns:
        float: Accuracy of the model.
        """
        predictions = self.predict(X)
        return np.mean(predictions == y)

# Create and train AdaBoost model
model = AdaBoost(n_estimators=50, learning_rate=1.0)
model.fit(X_train, y_train)

# Evaluate model
accuracy = model.score(X_test, y_test)
print(f"Accuracy: {accuracy:.3f}")